In [1]:
!pip install -U sentence-transformers
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/9a/06/e4ec2a321e57c03b7e9345d709d554a52c33760e5015fdff0919d9459af0/transformers-4.35.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00:00:01
  Obtaining dependency information for huggingface-hub>=0.4.0 from https://files.pythonhosted.org/packages/ef/b5/b6107bd65fa4c96fdf00e4733e2fe5729bb9e5e09997f63074bb43d3ab28/huggingface_hub-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10

In [3]:
import pandas as pd
import chromadb
from chromadb.api.types import Documents, Embeddings
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, util

In [60]:
# sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1")


In [4]:
df_dataset = pd.read_csv('data.csv')

In [7]:
df_dataset['input'] = df_dataset['leader'] + ' ' + df_dataset['user']

In [9]:
df = df_dataset[df_dataset['label'] == 1]

In [25]:
df

,leader,user,english_translation,assistant,english_translation.1,label,with_error,Note,input
0,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"Hola. Un café, por favor.",NaN,"Entiendo, un café.",NaN,1.0,NaN,default script pairs,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
1,¿Quieres leche y azúcar?,"Sí, leche y azúcar.",NaN,NaN,NaN,1.0,NaN,default script pairs,"¿Quieres leche y azúcar? Sí, leche y azúcar."
2,¿Cuántos azúcares?,Tres azúcares.,NaN,Leche y tres azúcares.,NaN,1.0,NaN,default script pairs,¿Cuántos azúcares? Tres azúcares.
5,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"Sí, quiero un café, por favor.",NaN,Un café.,NaN,1.0,NaN,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
6,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,un café,one cafe.,Un café.,NaN,1.0,NaN,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
7,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"Sí, quiero un cafés, por favor.",NaN,quieres un café.,NaN,1.0,NaN,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
8,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"una agua, favor.",NaN,quieres un agua.,NaN,1.0,NaN,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
9,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"dos café, por favor.",NaN,quieres dos cafés.,NaN,1.0,NaN,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
15,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"Sí, ¿puedo tomar un café con leche?","Yes, can I have a latte?",Seguro. un café con leche,NaN,1.0,0.0,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...
18,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...,"Sí, puedo tomo un café con leche.","Yes, can I have a latte?",Seguro. un café con leche,NaN,1.0,1.0,NaN,¡Hola! Bienvenido a la cafetería Brew Haven. ¿...


In [32]:
def create_chroma_db(qa_input, user_input, name):
  # get the client
  chroma_client = chromadb.Client()
  ## In-memory chroma with saving/loading to disk
  # chroma_client = chromadb.PersistentClient(path="/path/to/data")
  db = chroma_client.create_collection(name=name, embedding_function=sentence_transformer_ef)
  for i,d in enumerate(qa_input):
    db.add(
      documents= user_input[i],
      metadatas = {'lesson': d},
      ids=str(i)
    )
  return db

In [33]:
db = create_chroma_db(df['input'].to_list(), df['user'].to_list(), "langbot")

In [46]:
results = db.query(
    query_texts=["Un café, por favor."],
    n_results=1
)


In [47]:
results

{'ids': [['0']],
 'distances': [[0.15042023360729218]],
 'metadatas': [[{'input': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber? Hola. Un café, por favor.'}]],
 'embeddings': None,
 'documents': [['Hola. Un café, por favor.']],
 'uris': None,
 'data': None}

In [38]:
results['distances'][0]

[1.1485252380371094]

In [40]:
def create_chroma_db(documents, user, name):
  # get the client
  chroma_client = chromadb.Client()
  ## In-memory chroma with saving/loading to disk
  # chroma_client = chromadb.PersistentClient(path="/path/to/data")
  db = chroma_client.create_collection(name=name, embedding_function=sentence_transformer_ef)
  for i,d in enumerate(documents):
    db.add(
      documents= d,
      metadatas = {'lesson': d},
      ids=str(i)
    )
  return db

In [41]:
db1 = create_chroma_db(df['input'].to_list(), df['user'].to_list(), "lang-bot-test")

In [58]:
results = db1.query(
    query_texts=["¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber? Hola."],
    n_results=1,
    where = {'lesson':'1' }
)

In [59]:
results

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None}

In [ ]:
def create_chroma_db(documents, user, name):
  # get the client
  chroma_client = chromadb.Client()
  ## In-memory chroma with saving/loading to disk
  # chroma_client = chromadb.PersistentClient(path="/path/to/data")
  db = chroma_client.create_collection(name=name, embedding_function=sentence_transformer_ef)
  for i,d in enumerate(documents):
    db.add(
      documents= d,
      metadatas = {'input': d},
      ids=str(i)
    )
  return db

In [61]:
def create_chroma_db(qa_input, user_input, name):
  # get the client
  chroma_client = chromadb.Client()
  ## In-memory chroma with saving/loading to disk
  # chroma_client = chromadb.PersistentClient(path="/path/to/data")
  db = chroma_client.create_collection(name=name, embedding_function=sentence_transformer_ef)
  for i,d in enumerate(qa_input):
    db.add(
      documents= d,
      metadatas = {'lesson': d},
      ids=str(i)
    )
  return db

In [74]:
question = [ '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?',
'¿Quieres leche y azúcar?',
'¿Cuántos azúcares?',
'¿Quieres comida? ¿Sándwich o algo dulce?',
'¿Pagas con tarjeta o en efectivo?' ]

In [63]:
db2 = create_chroma_db(df['leader'].to_list(), df['user'].to_list(), "lang-bot-test")

In [68]:
results = db2.query(
    query_texts=["Un café, por favor."],
    n_results=1
)

In [69]:
results

{'ids': [['6']],
 'distances': [[1.0535516738891602]],
 'metadatas': [[{'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'}]],
 'embeddings': None,
 'documents': [['¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?']],
 'uris': None,
 'data': None}

In [72]:
results = db2.query(
    query_texts=["Hola."],
    n_results=10
)

In [73]:
results

{'ids': [['0', '3', '4', '5', '6', '7', '8', '9', '10', '11']],
 'distances': [[1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975,
   1.1834352016448975]],
 'metadatas': [[{'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¡Hola! Bienvenido a la cafeterí

In [77]:
db3 = create_chroma_db(question, question, "lang-bot-test4")

In [83]:
results = db3.query(
    query_texts=["En efectivo."],
    n_results=5
)

In [84]:
results

{'ids': [['4', '0', '2', '3', '1']],
 'distances': [[0.8087770342826843,
   1.2825713157653809,
   1.3265247344970703,
   1.4157676696777344,
   1.4256945848464966]],
 'metadatas': [[{'lesson': '¿Pagas con tarjeta o en efectivo?'},
   {'lesson': '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?'},
   {'lesson': '¿Cuántos azúcares?'},
   {'lesson': '¿Quieres comida? ¿Sándwich o algo dulce?'},
   {'lesson': '¿Quieres leche y azúcar?'}]],
 'embeddings': None,
 'documents': [['¿Pagas con tarjeta o en efectivo?',
   '¡Hola! Bienvenido a la cafetería Brew Haven. ¿Algo de beber?',
   '¿Cuántos azúcares?',
   '¿Quieres comida? ¿Sándwich o algo dulce?',
   '¿Quieres leche y azúcar?']],
 'uris': None,
 'data': None}